# Homography Regression Qualitative Evaluation
A homography regression network that was trained on synthetically generated camera motion data is evaluated on data where camera motion occurs naturally.

## Load Model

In [4]:
import os
import torch
from utils.io import load_yaml
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

# load best model\n",
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone/version_0'
configs = load_yaml(os.path.join(prefix, 'configs.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=19.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()

Running with CUDA backend.


In [5]:
# create dataset and evaluate model on
#   - with motion dataset
#   - cholec80
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from kornia import warp_perspective, tensor_to_image
from torchvision.transforms import ToTensor
from datasets import ImageSequenceDataset
from utils.processing import image_edges, four_pt_to_matrix_homography_representation

## With Camera Motion - DaVinci

In [6]:
# motion dataset
prefix = '/media/martin/Samsung_T5/data/endoscopic_data/camera_motion_separated_png/with_camera_motion'
df_name = 'log_with_camera_motion_seq_len_2.pkl'
df = pd.read_pickle(os.path.join(prefix, df_name))
ds = ImageSequenceDataset(df, prefix, ToTensor())

for img_seq in ds:
    img_seq[0], img_seq[1] = img_seq[0].to(device), img_seq[1].to(device)
    duv = model(img_seq[0].unsqueeze(0), img_seq[1].unsqueeze(0))

    uv = image_edges(img_seq[0].unsqueeze(0))
    H = four_pt_to_matrix_homography_representation(uv, duv)

    wrp = warp_perspective(img_seq[0].unsqueeze(0), torch.inverse(H), img_seq[0].shape[-2:])
    
    wrp = tensor_to_image(wrp)
    img0 = tensor_to_image(img_seq[0])
    img1 = tensor_to_image(img_seq[1])

    cv2.imshow('img1', img1)
    cv2.imshow('wrp', wrp)
    cv2.imshow('L1 img0, img1', abs(img0-img1))
    cv2.imshow('L1 img1, wrp', abs(img1-wrp))
    cv2.waitKey()

## With Camera Motion - Cholec80
### Image Shapes

In [1]:
# cholec80, analyze image shape
import os
import cv2
from utils.io import load_yaml

cholec_configs = load_yaml('configs/cholec80_transforms.yml')

database_prefix = '/media/martin/Samsung_T5/data/endoscopic_data'

for database in cholec_configs['databases']:
    path = os.path.join(database_prefix, database['prefix'])
    for file in database['videos']['files']:
        path = os.path.join(database_prefix, database['prefix'], database['videos']['prefix'], file)
        vc = cv2.VideoCapture(path)

        _, img = vc.read()
        cv2.imshow('img', img)
        cv2.waitKey()

### Homography Estimation

In [ ]:
# cholec80, evaluate model
import pandas as pd
from torchvision.transforms import ToTensor
from datasets import ImageSequenceDataset

prefix = '/media/martin/Samsung_T5/data/endoscopic_data/tmp'
df_name = 'cholec80_seq_len_2.pkl'
df = pd.read_pickle(os.path.join(prefix, df_name))

ds = ImageSequenceDataset(df, prefix, ToTensor())

for img_seq in ds:
    img_seq[0], img_seq[1] = img_seq[0].to(device), img_seq[1].to(device)
    duv = model(img_seq[0].unsqueeze(0), img_seq[1].unsqueeze(0))

    uv = image_edges(img_seq[0].unsqueeze(0))
    H = four_pt_to_matrix_homography_representation(uv, duv)

    wrp = warp_perspective(img_seq[0].unsqueeze(0), torch.inverse(H), img_seq[0].shape[-2:])

    wrp = tensor_to_image(wrp)
    img0 = tensor_to_image(img_seq[0])
    img1 = tensor_to_image(img_seq[1])

    cv2.imshow('img1', img1)
    cv2.imshow('wrp', wrp)
    cv2.imshow('L1 img0, img1', abs(img0-img1))
    cv2.imshow('L1 img1, wrp', abs(img1-wrp))
    cv2.waitKey()